In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import mne
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
# 1. Data Loading and Preprocessing
class EEGDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.LongTensor(labels)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [15]:
file_path = '/home/technoshr370/Downloads/Dataset/ds003751-1.0.6/sub-mit003/eeg/sub-mit003_task-Emotion_eeg_preprocessed.edf'

In [17]:

def load_edf_data(file_path, preprocess=True):
    """
    Load EDF file using MNE and preprocess the data
    
    Args:
        file_path (str): Path to the EDF file
        preprocess (bool): Whether to apply preprocessing
    
    Returns:
        numpy.ndarray: Preprocessed EEG data
    """
    # Read EDF file
    raw = mne.io.read_raw_edf(file_path, preload=True)
    
    if preprocess:
        # Basic preprocessing steps
        raw.filter(l_freq=1, h_freq=50)  # Bandpass filter
        raw.notch_filter(50)  # Remove power line noise
    
    # Extract data as numpy array
    data = raw.get_data()
    
    return data


In [18]:
def segment_data(data, window_size=500, stride=250):
    """
    Segment continuous EEG data into fixed-length windows
    
    Args:
        data (numpy.ndarray): Raw EEG data
        window_size (int): Size of each segment
        stride (int): Overlap between segments
    
    Returns:
        numpy.ndarray: Segmented EEG data
    """
    segments = []
    for i in range(0, data.shape[1] - window_size, stride):
        segment = data[:, i:i+window_size]
        segments.append(segment)
    
    return np.array(segments)

In [19]:
# 2. Autoencoder Architecture
class EEGAutoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=32):
        super().__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, encoding_dim)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded


In [20]:
# 3. LSTM Network for EEG Classification
class EEGClassifierLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=0.3)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM forward
        out, _ = self.lstm(x, (h0, c0))
        
        # Take the last time step
        out = self.fc(out[:, -1, :])
        return out

In [21]:
# 4. Training Workflow
def train_autoencoder(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        optimizer.zero_grad()
        
        # Flatten input
        x = batch[0].view(batch[0].size(0), -1).to(device)
        
        # Forward pass
        encoded, decoded = model(x)
        loss = criterion(decoded, x)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [22]:
def train_lstm_classifier(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch_data, batch_labels in dataloader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)

In [25]:
# 5. Main Workflow Execution
def main():
    # Hyperparameters
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 64
    EPOCHS = 50
    LEARNING_RATE = 0.001
    
    # Load EDF data
    edf_file_path = 'path/to/your/eeg_data.edf'
    raw_data = load_edf_data(edf_file_path)
    
    # Segment data
    segmented_data = segment_data(raw_data)
    
    # Prepare data for autoencoder
    X_train, X_test = train_test_split(segmented_data, test_size=0.2)
    
    # Flatten and standardize
    scaler = StandardScaler()
    X_train_flat = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
    X_test_flat = scaler.transform(X_test.reshape(X_test.shape[0], -1))
    
    # DataLoaders
    train_dataset = torch.utils.data.TensorDataset(
        torch.FloatTensor(X_train_flat), 
        torch.zeros(len(X_train_flat), dtype=torch.long)
    )
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # Initialize Autoencoder
    input_dim = X_train_flat.shape[1]
    autoencoder = EEGAutoencoder(input_dim).to(DEVICE)
    
    # Autoencoder Training
    ae_criterion = nn.MSELoss()
    ae_optimizer = optim.Adam(autoencoder.parameters(), lr=LEARNING_RATE)
    
    for epoch in range(EPOCHS):
        ae_loss = train_autoencoder(autoencoder, train_loader, ae_criterion, ae_optimizer, DEVICE)
        print(f"Autoencoder Epoch {epoch+1}, Loss: {ae_loss}")
    
    # Extract compressed representations
    compressed_data = []
    with torch.no_grad():
        for batch in train_loader:
            x = batch[0].view(batch[0].size(0), -1).to(DEVICE)
            encoded, _ = autoencoder(x)
            compressed_data.append(encoded.cpu().numpy())
    
    compressed_data = np.vstack(compressed_data)

    # Prepare LSTM data
    lstm_dataset = EEGDataset(compressed_data, np.random.randint(0, 2, len(compressed_data)))
    lstm_loader = DataLoader(lstm_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # LSTM Training
    lstm_criterion = nn.CrossEntropyLoss()
    lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE)
    
    for epoch in range(EPOCHS):
        lstm_loss = train_lstm_classifier(lstm_model, lstm_loader, lstm_criterion, lstm_optimizer, DEVICE)
        print(f"LSTM Epoch {epoch+1}, Loss: {lstm_loss}")
    
    # Save models
    torch.save(autoencoder.state_dict(), 'eeg_autoencoder.pth')
    torch.save(lstm_model.state_dict(), 'eeg_lstm_classifier.pth')



In [26]:

if __name__ == "__main__":
    main()

Extracting EDF parameters from /home/technoshr370/Downloads/Code/path/to/your/eeg_data.edf...
EDF file detected


FileNotFoundError: [Errno 2] No such file or directory: '/home/technoshr370/Downloads/Code/path/to/your/eeg_data.edf'